# A/B Testing the Udacity Website

## Exercise 1
Begin by importing Udacity’s data on user behavior by going to http://www.github.com/nickeubank/MIDS_Data/ and using the udacity_AB_testingfolder, or by clicking here. Note that there are TWO datasets for this test – one for the control data (users who saw the original design), and one for treatment data (users who saw the experimental design). Udacity decided to show their test site to 1/2 of visitors, so there are roughly the same number of users appearing in each dataset (though this is not a requirement of AB tests).

In [2]:
import pandas as pd

In [4]:
control_data = pd.read_csv("./data/control_data.csv")
experiment_data = pd.read_csv("./data/experiment_data.csv")

In [5]:
experiment_data.head()

,Date,Pageviews,Clicks,Enrollments,Payments
0,"Sat, Oct 11",7716,686,105.0,34.0
1,"Sun, Oct 12",9288,785,116.0,91.0
2,"Mon, Oct 13",10480,884,145.0,79.0
3,"Tue, Oct 14",9867,827,138.0,92.0
4,"Wed, Oct 15",9793,832,140.0,94.0


In [6]:
control_data.head()

,Date,Pageviews,Clicks,Enrollments,Payments
0,"Sat, Oct 11",7723,687,134.0,70.0
1,"Sun, Oct 12",9102,779,147.0,70.0
2,"Mon, Oct 13",10511,909,167.0,95.0
3,"Tue, Oct 14",9871,836,156.0,105.0
4,"Wed, Oct 15",10014,837,163.0,64.0


## Exercise 2
Explore the data. Can you identifying the unit of observation of the data (e.g. what is represented by each row)?

> Here, each row indicates the performance of a course on a specific day. For each row, it contains five properties (columns).

> - Date:  Date
> - Pageviews:  Number of unique **cookies** to view the course overview page that day.
> - Clicks: Number of **unique cookies** to click the course overview page that day.
> - Enrollments: Number of **user-ids to** enroll in the free trial that day.
> - Payments: Number of **user-ids** who who enrolled on that day to remain enrolled for 14 days and thus make a payment.

> One thing two notice is that unite of pageviews and clicks is different from enrollments and payments. The first two properties' unit is unique cookies while the last two properties are user-ids.

## Exercise 3
The easiest way to analyze this data is to stack it into a single dataset where each observation is a day-treatment-arm (so you should end up with two rows per day, one for those who are in the treated groups, and one for those who were in the control group). Note that currently nothing in the data identifies whether a given observation is a treatment group observation or a control group observation, so you’ll want to make sure to add a “treatment” indicator variable.

The variables in the data are:

- Pageviews: number of unique users visiting homepage
- Clicks: number of those users clicking “Start Free Trial”
- Enrollments: Number of people enrolling in trial
- Payments: Number of people who eventually pay for the service

In [35]:
control_data['treatment'] = 0
experiment_data['treatment'] = 1

data = pd.concat([control_data, experiment_data]).sort_values("Date")

# Convert Date to standard np datetime, assume default year is 2017
data['Date'] = pd.to_datetime(data['Date'] , format='%a, %b %d') \
                 .apply(lambda x: x.replace(year=2017)) 

data.sort_values(['Date', 'treatment'], inplace = True)
data.head(n=6)

,Date,Pageviews,Clicks,Enrollments,Payments,treatment
0,2017-10-11,7723,687,134.0,70.0,0
0,2017-10-11,7716,686,105.0,34.0,1
1,2017-10-12,9102,779,147.0,70.0,0
1,2017-10-12,9288,785,116.0,91.0,1
2,2017-10-13,10511,909,167.0,95.0,0
2,2017-10-13,10480,884,145.0,79.0,1


#### Check is there unmatched record 

In [40]:
sum(data.groupby("Date")["Pageviews"].count() == 1)

0

0 means all records are pairs wised, one comes from the control group, and another comes from the treatment group. 

# Exercise 4
Given the outcomes of interest to Udacity, what outcomes do you want to measure? (In the language of the Potential Outcomes Framework, what are your Y variables?). Add these to your data.

> The Udacity is interested in Net Conversion, which is the ratio of Payments and Number of Clicks on Free Trial Button.

In [44]:
data['NetConv'] = data['Payments'] / data['Clicks'] 
data.head()

,Date,Pageviews,Clicks,Enrollments,Payments,treatment,NetConv
0,2017-10-11,7723,687,134.0,70.0,0,0.101892
0,2017-10-11,7716,686,105.0,34.0,1,0.049563
1,2017-10-12,9102,779,147.0,70.0,0,0.089859
1,2017-10-12,9288,785,116.0,91.0,1,0.115924
2,2017-10-13,10511,909,167.0,95.0,0,0.104510


## Exercise 5
Whenever you are working with experimental data, the first thing you want to do is verify that users actually were randomly sorted into the two arms of the experiment. In this data, half of users were supposed to be shown the old version of the site and half were supposed to see the new version.

Pageviews tells you how many unique users visited the welcome site we are experimenting on. Pageviews is what is sometimes called an “invariant” variable, meaning that it shouldn’t vary across treatment arms – after all, people have to visit the site before they get a chance to see the treatment, so there’s no way that being assigned to treatment or control should affect the number of pageviews assigned to each group.

“Invariant” variables are also an example of what are known as a “pre-treatment” variable, because pageviews are determined before users are manipulated in any way. That makes it analogous to gender or age in experiments where you have demographic data – a person’s age and gender are determined before they experience any manipulations, so the value of any pre-treatment attributes should be the same across the two arms of our experiment. This is what is called “checking for balance.” If pre-treatment attributes aren’t balanced, then we know our attempt to randomly assign people to different groups failed.

To test the quality of the randomization, calculate the average number of pageviews for the treated group and for the control group. Do they look similar?

## Exercise 6
“Similar” is a tricky concept – obviously, we expect some differences across groups since users were randomly divided across treatment arms. The question is whether the differences between groups are larger than we’d expect to emerge given our random assignment process. To evaluate this, let’s use a ttest to test the statistical significance of the differences we see.

If you’re using R, you can just use the t.test function.

If you’re using Python, you can use the ttest function from scipy, which you can import as from scipy.stats import ttest_ind.

Note: Remember that scipy functions don’t accept pandas objects, so you have to pass the numpy vectors underlying your data with the .values operator (e.g. df.my_column.values).

Does the difference in pageviews look statistically significant?

## Exercise 7
Pageviews is not the only pre-treatment variable in this data. What other measure is pre-treatment? Review the description of the experiment if you’re not sure.

## Exercise 8
Check if the other pre-treatment variable is also balanced.

## Exercise 9
Now that we’ve established we have good balance (meaning we think randomization was likely successful), we can evaluate the effects of the experiment. Test whether the two metrics you picked have different average values in the control group and treatment group. Because we’ve randomized, this is a consistent estimate of the Average Treatment Effect of Udacity’s website change.

Did Udacity achieve their goal?

Note: You may discover some issues with your data. Can you figure out what’s going on, and adjust?

## Exercise 10
One of the magic things about experiments is that all you have to do is compare averages to get an average treatment effect. However, you can do other things to try and increase the statistical power of your experiments, like add controls in a linear regression model.

As you likely know, a bivariate regression is exactly equivalent to a t-test, so let’s start by re-estimating the effect of treatment on payments-per-click using a linear regression. Can you replicate the results from your t-test? They shouldn’t just be close – they should be numerically equivalent (i.e. exactly the same to the limits of floating point number precision).

## Exercise 11
Now add indicator variables for the day of each observation. Do the standard errors on your treatment variable change? If so, in what direction?

You should have found that your standard errors decreased by about 20% – this is why, although just comparing means works, if you have additional variables you should add them as covariates in your analysis. Moreover, in other settings you may find this effect is even larger – the date indicators we added to our data are perfectly balanced between treatment and control, so we aren’t adding a lot of data to the model by adding them as variables. As we’ll see in later exercises, adding variables like “gender” or “age” (which will never be perfectly balanced across treatment and control) will help even more.

## Exercise 12
Given your results, what would you tell Udacity about their trial?

## Exercise 13
As a last exercise, instead of adding indicators for each date, add indicators for day of the week (e.g. Monday, Tuesday, etc.).

(This is just for data manipulation practice!)